In [162]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit
import torch
import torchvision

In [163]:
# set cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Dataloader

In [164]:
class activityData(torch.utils.data.Dataset):
    '''
    Class for activity data.
    '''
    def __init__(self, url:str):
        data = np.load(url)
        self.x = torch.from_numpy(data['X'])
        self.y = torch.from_numpy(data['y'])
        self.n_samples = self.x.shape[0]
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_samples

In [165]:
batchsize = 128

# create datasets and dataloaders
dataTrain = activityData(url='train.npz')
dataTest = activityData(url="test.npz")
loaderTrain = torch.utils.data.DataLoader(dataset=dataTrain, batch_size=batchsize, shuffle=True)
loaderTest = torch.utils.data.DataLoader(dataset=dataTest, batch_size=batchsize, shuffle=False)

# Define model of CNN

In [166]:
loaderTrain.dataset.x[0].shape, loaderTrain.dataset.y[0].shape

(torch.Size([43, 13]), torch.Size([]))

# Functions for training model

In [167]:
def calcOutputDim(Input:int, kernel:int, padding:int, stride:int):
    '''
    Calculates output width of a convolutional layer.
    '''
    return int((Input - kernel + 2*padding)/stride + 1)

In [159]:
def train(model, epochs:int, loader, optimizer, criterion):
    model.train()
    loss_hist = []
    loss = 0
    lossepoch = 0

    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(loader):
            data = data.unsqueeze(1).float()
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            loss_hist.append(loss.item())
            lossepoch += loss.item()

            if epoch + 1 == epochs and loss_hist[-2] > loss_hist[-1]:
                best_model = model.state_dict()

        print("loss", lossepoch)
        lossepoch = 0
    
    model = model.load_state_dict(best_model)

    return loss_hist, model

In [181]:
def evalTest(model, testLoader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in testLoader:
            data = data.unsqueeze(1).float()
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(testLoader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(testLoader.dataset),
        100. * correct / len(testLoader.dataset)))

# Model with 1 convolutional layer

In [160]:
# define model
class CNN(torch.nn.Module):
    def __init__(self, poolingFunction, bnFunction, activationFunction, kernel=None, stride=None, padding=None):
        super(CNN, self).__init__()
        
        # building network with specified number of layers, poolingFunction and bnFunction
        self.network = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=2),
            poolingFunction(kernel_size=2, stride=2, padding=1),
            bnFunction(num_features=16),
            activationFunction(),
        
            # torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            # poolingFunction(kernel_size=2, stride=2, padding=1),
            # bnFunction(num_features=32),
            # activationFunction(),

            # torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            # poolingFunction(kernel_size=2, stride=2, padding=1),
            # bnFunction(num_features=64),
            # activationFunction(),
        )

        self.linStack = torch.nn.Sequential(
            torch.nn.Linear(in_features=2944, out_features=1500),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(num_features=1500),
            torch.nn.Linear(in_features=1500, out_features=7),
        )

    def forward(self, x):
        x = self.network(x)
        x = torch.nn.Flatten(1)(x)
        x = self.linStack(x)
        return x


In [161]:
model = CNN(poolingFunction=torch.nn.MaxPool2d, bnFunction=torch.nn.BatchNorm2d, activationFunction=torch.nn.ReLU)
loss_hist, model = train(model=model, epochs=20, loader=loaderTrain, optimizer=torch.optim.Adam(model.parameters(), lr=1e-3), criterion=torch.nn.CrossEntropyLoss())
evalTest(model=model, testLoader=loaderTest, criterion=torch.nn.CrossEntropyLoss())
torch.save(model.state_dict(), "model1.pt")
plt.plot(loss_hist)

KeyboardInterrupt: 

# Model with 2 convolutional layer

In [147]:
# define model
class CNN(torch.nn.Module):
    def __init__(self, poolingFunction, bnFunction, activationFunction, kernel=None, stride=None, padding=None):
        super(CNN, self).__init__()

        # building network with specified number of layers, poolingFunction and bnFunction
        self.network = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=2),
            poolingFunction(kernel_size=2, stride=2, padding=1),
            bnFunction(num_features=16),
            activationFunction(),
        
            torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            poolingFunction(kernel_size=2, stride=2, padding=1),
            bnFunction(num_features=32),
            activationFunction(),

            # torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            # poolingFunction(kernel_size=2, stride=2, padding=1),
            # bnFunction(num_features=64),
            # activationFunction(),
        )

        self.linStack = torch.nn.Sequential(
            torch.nn.Linear(in_features=1920, out_features=1000),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(num_features=1000),
            torch.nn.Linear(in_features=1000, out_features=7),
        )

    def forward(self, x):
        x = self.network(x)
        x = torch.nn.Flatten(1)(x)
        x = self.linStack(x)
        return x


In [148]:
model = CNN(poolingFunction=torch.nn.MaxPool2d, bnFunction=torch.nn.BatchNorm2d, activationFunction=torch.nn.ReLU)
loss_hist, model = train(model=model, epochs=20, loader=loaderTrain, optimizer=torch.optim.Adam(model.parameters(), lr=1e-3), criterion=torch.nn.CrossEntropyLoss())
evalTest(model=model, testLoader=loaderTest, criterion=torch.nn.CrossEntropyLoss())
torch.save(model.state_dict(), "model2.pt")
plt.plot(loss_hist)

KeyboardInterrupt: 

# Model with 3 convolutional layer

In [151]:
# define model
class CNN(torch.nn.Module):
    def __init__(self, poolingFunction, bnFunction, activationFunction, kernel=None, stride=None, padding=None):
        super(CNN, self).__init__()

        # building network with specified number of layers, poolingFunction and bnFunction
        self.network = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=2),
            poolingFunction(kernel_size=2, stride=2, padding=1),
            bnFunction(num_features=16),
            activationFunction(),
        
            torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            poolingFunction(kernel_size=2, stride=2, padding=1),
            bnFunction(num_features=32),
            activationFunction(),

            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            poolingFunction(kernel_size=2, stride=2, padding=1),
            bnFunction(num_features=64),
            activationFunction(),
        )

        self.linStack = torch.nn.Sequential(
            torch.nn.Linear(in_features=1344, out_features=672),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(num_features=672),
            torch.nn.Linear(in_features=672, out_features=7),
        )

    def forward(self, x):
        x = self.network(x)
        x = torch.nn.Flatten(1)(x)
        x = self.linStack(x)
        return x


In [152]:
model = CNN(poolingFunction=torch.nn.MaxPool2d, bnFunction=torch.nn.BatchNorm2d, activationFunction=torch.nn.ReLU)
loss_hist, model = train(model=model, epochs=20, loader=loaderTrain, optimizer=torch.optim.Adam(model.parameters(), lr=1e-3), criterion=torch.nn.CrossEntropyLoss())
evalTest(model=model, testLoader=loaderTest, criterion=torch.nn.CrossEntropyLoss())
torch.save(model.state_dict(), "model3.pt")
plt.plot(loss_hist)

KeyboardInterrupt: 